In [ ]:
%matplotlib inline
from lasagne import layers
from lasagnekit.misc.plot_weights import dispims_color
from skimage.io import imsave
import sys, os
sys.path.append(os.getcwd()+"/..")
from tools.system.imgtovideo import imgs_to_video
from data import load_data
import matplotlib.pyplot as plt
import numpy as np
import theano
import theano.tensor as T
from lasagne import layers as L
from lasagnekit.misc.plot_weights import dispims_color, tile_raster_images
import pandas as pd

from tqdm import tqdm
import base64
import json

from skimage.io import imread, imsave
from skimage.transform import resize

from IPython.display import HTML, Image

from model import model102, model103
from tools.brushstroke.common import (
    resize_set, load_model, get_bias, 
    get_scale, build_brush_func,
    build_encode_func,
    build_image_to_code_func,
    build_code_to_image,
    to_grid_of_images,
    seq_to_video,
    embed_video,
    disp_grid,
    prop_uniques,
    hash_array,
    normalize,
    sigmoid,
    build_pointer_images)
from helpers import GenericBrushLayer

In [ ]:
n_steps = 2
nb_features = 100
inp = layers.InputLayer((None, n_steps, nb_features))
patches = np.ones((1, 1, 4, 4))
patches = patches.astype(np.float32)

def nm(x):
    x = x - x.min(axis=(2,3), keepdims=True)
    x = x / (x.max(axis=(2,3), keepdims=True)+1e-10)
    return x
c = [
    [1, 0, 0],
    [0, 1, 0]
]
c = np.array(c)
c = c.astype(np.float32)

brush = GenericBrushLayer(
    inp,
    16, 16,
    n_steps=n_steps,
    patches=patches,
    col='grayscale',
    return_seq=True,
    reduce_func=lambda x,y:x,
    to_proba_func=T.nnet.softmax,
    normalize_func=lambda x:x,
    x_sigma=0.1,
    y_sigma=0.1,
    x_stride=1,
    y_stride=1,
    patch_index=0,
    color=[1.],
    color_min=0,
    color_max=1,
    x_min=0,
    x_max=16,
    y_min=0,
    y_max=16,
    w_left_pad=0,
    w_right_pad=0,
    h_left_pad=0,
    h_right_pad=0,
    stride_normalize=True,
    coords='discrete',
    eps=0)

In [ ]:
X = T.tensor3()
fn = theano.function([X], layers.get_output(brush, X))
print(brush.assign_)

In [ ]:
coord = np.zeros((9, 2, nb_features))
coord[:, 0, 11]=1000

coord[:, 1, 4]=1000
coord[:, 1, 4]=1000

"""
coord[:, 0, 0] = 0.3
coord[:, 0, 1] = 0.5

coord[:, 1, 0] = 0.7
coord[:, 1, 1] = 0.5
"""

coord = coord.astype(np.float32)
y = fn(coord)
a, b = y[:, 0], y[:, 1]
y=a#+b*(a< (1e-5))
img = disp_grid(y, border=1, bordercolor=(0.3,0,0))
imsave('out.png', img)
Image('out.png', width=400, height=400)

In [ ]:
print(b.min(), b.max())

In [ ]:
from data import load_data

In [ ]:
kw = {
   "pipeline": [
        {"name": "toy", "params": {"nb": 100, "w": 16, "h": 16, "pw": 4, "ph": 4, "nb_patches": 2}},
        {"name": "shuffle", "params": {}},
        {"name": "normalize_shape", "params": {}},
        {"name": "divide_by", "params": {"value": 255}},
        {"name": "order", "params": {"order": "th"}}
    ]
}
data = load_data('loader', **kw)
data.load()
X = data.X

In [ ]:
data.load()
print(X.min(), X.max())
plt.imshow(disp_grid(X, border=1, bordercolor=(0.3,0,0)))

In [ ]:
lays = model103(w=16,h=16,c=1,n_steps=2)
X = T.tensor4()
fn = theano.function([X], layers.get_output(lays['raw_output'], X))

In [ ]:
br = theano.function([X], layers.get_output(lays['brush'], X))

In [ ]:
out = theano.function([X], layers.get_output(lays['output'], X))

In [ ]:
out(data.X).max()